In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np


In [2]:
# Read in the input data
data = pd.read_csv('heart.csv')

In [3]:
# Split the data into features and target variable
x = data.drop(['HeartDisease'], axis=1)
y = data['HeartDisease']

In [4]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Convert categorical data into numerical data
cat_cols = x_train.select_dtypes('object').keys().tolist()
x_train = pd.get_dummies(data=x_train, columns=cat_cols, drop_first=True)
x_test = pd.get_dummies(data=x_test, columns=cat_cols, drop_first=True)

In [5]:
# Standardize the data
x_train_cols = x_train.columns
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_test = pd.DataFrame(x_test, columns=x_train_cols)

In [6]:
# Fit a Logistic Regression model
model = LogisticRegression(solver='lbfgs')
model.fit(x_train, y_train)

# Make predictions on the test set
y_pred_proba = model.predict_proba(x_test)
y_pred = np.argmax(y_pred_proba, axis=1)

# Print the accuracy score
print("Accuracy:", accuracy_score(y_test, y_pred))

# Print other evaluation metrics
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))


Accuracy: 0.8804347826086957
Precision: 0.9171974522292994
Recall: 0.8780487804878049
F1-Score: 0.897196261682243


In [7]:
import ipywidgets as widgets
from IPython.display import display

# Create widgets for the input features
age_text = widgets.IntText(value=0, description='Age:')
sex_dropdown = widgets.Dropdown(options=['Male','Female'],value='Male',description='Sex:')
cp_dropdown = widgets.Dropdown(options=[1,2,3,4],value=1,description='Chest Pain Type:')
trestbps_text = widgets.IntText(value=0, description='Resting Blood Pressure:')
chol_text = widgets.IntText(value=0, description='Cholestoral:')
fbs_dropdown = widgets.Dropdown(options=[0,1],value=0,description='Fasting Blood Sugar:')
restecg_dropdown = widgets.Dropdown(options=[0,1,2],value=0,description='Resting Electrocardiographic Results:')
thalach_text = widgets.IntText(value=0, description='Maximum Heart Rate Achieved:')
exang_dropdown = widgets.Dropdown(options=[0,1],value=0,description='Exercise Induced Angina:')
oldpeak_text = widgets.FloatText(value=0, description='ST Depression Induced by Exercise:')
slope_dropdown = widgets.Dropdown(options=[1,2,3],value=1,description='Slope of the Peak Exercise ST Segment:')
ca_text = widgets.IntText(value=0, description='Number of Major Vessels Colored by Flourosopy:')
thal_dropdown = widgets.Dropdown(options=['Normal','Fixed defect','Reversible defect'],value='Normal',description='Thalassemia:')

In [8]:
# Create a button widget for making predictions
button = widgets.Button(description="Predict")

# Create widgets for displaying the output
disease_label = widgets.Label()
acc_label = widgets.Label()

In [9]:
import pandas as pd
# Define a function that maps the input values to numerical values
def map_inputs():
    sex_map = {'Male': 1, 'Female': 0}
    cp_map = {1: 1, 2: 2, 3: 3, 4: 4}
    fbs_map = {0: 0, 1: 1}
    restecg_map = {0: 0, 1: 1, 2: 2}
    exang_map = {0: 0, 1: 1}
    slope_map = {1: 1, 2: 2, 3: 3}
    thal_map = {'Normal': 0, 'Fixed defect': 1, 'Reversible defect': 2}
    
    x_test_temp = pd.DataFrame(columns=['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS', 'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope', 'CA', 'Thal'])
    x_test_temp = x_test_temp.append({'Age': age_text.value, 'Sex': sex_map[sex_dropdown.value], 'ChestPainType': cp_map[cp_dropdown.value], 'RestingBP': trestbps_text.value, 'Cholesterol': chol_text.value, 'FastingBS': fbs_map[fbs_dropdown.value], 'RestingECG': restecg_map[restecg_dropdown.value], 'MaxHR': thalach_text.value, 'ExerciseAngina': exang_map[exang_dropdown.value], 'Oldpeak': oldpeak_text.value, 'ST_Slope': slope_map[slope_dropdown.value], 'CA': ca_text.value, 'Thal': thal_map[thal_dropdown.value]}, ignore_index=True)
    return x_test_temp



In [10]:
def predict(b):
    x_test_temp = map_inputs()
    x_test_temp = pd.get_dummies(data=x_test_temp, columns=cat_cols, drop_first=True)
    x_test_temp = x_test_temp.reindex(columns=x_train_cols, fill_value=0)
    x_test_temp = scaler.transform(x_test_temp)
    y_pred = model.predict(x_test_temp)
    # predict the class of the input sample
    # pick only one sample from the test data
    x_test_temp = x_test.iloc[0].values.reshape(1, -1)
    y_test_temp = y_test.iloc[0]

    pred = model.predict(x_test_temp)
    disease_label.value = 'Predicted Heart Disease: ' + str(pred[0])

    # compare the predicted class with the true label of the input sample
    acc = accuracy_score([y_test_temp], pred)
    acc_label.value = 'Accuracy: ' + str(acc)


In [11]:
# Create a button widget for making predictions
button = widgets.Button(description="Predict")

# Assign the function to the on_click event
button.on_click(predict)


In [12]:
display(age_text, sex_dropdown, cp_dropdown, trestbps_text, chol_text, fbs_dropdown, restecg_dropdown, thalach_text, exang_dropdown, oldpeak_text, slope_dropdown, ca_text, thal_dropdown, button, disease_label, acc_label)

IntText(value=0, description='Age:')

Dropdown(description='Sex:', options=('Male', 'Female'), value='Male')

Dropdown(description='Chest Pain Type:', options=(1, 2, 3, 4), value=1)

IntText(value=0, description='Resting Blood Pressure:')

IntText(value=0, description='Cholestoral:')

...

IntText(value=0, description='Number of Major Vessels Colored by Flourosopy:')

Dropdown(description='Thalassemia:', options=('Normal', 'Fixed defect', 'Reversible defect'), value='Normal')

Button(description='Predict', style=ButtonStyle())

Label(value='')

Label(value='')